# Deep500 without recipes

This tutorial shows how to construct and run a benchmark manually, without the recipe interface.
We will train CIFAR-10 for 5 epochs.

First, import the packages for Deep500, datasets, and networks:

In [1]:
import deep500 as d5
from deep500 import datasets as d5ds
from deep500 import networks as d5nt

## Model

We now need to create a DNN (ResNet-20), which requires the dimensions of the dataset samples. The first step is to obtain the sample shape and loss function information from CIFAR-10, as well as set a minibatch size (due to ONNX static sizes):

In [2]:
shape = d5ds.dataset_shape('cifar10')
loss = d5ds.dataset_loss('cifar10')
batch_size = 64
epochs = 5
print(shape, loss)

(10, 3, 32, 32) <class 'deep500.utils.onnx_interop.losses.SoftmaxCrossEntropy'>


The output of `dataset_shape` is organized as (number of classes, sample dimensions...).

The next part is to construct the model. In `deep500.networks`, we provide a built-in generator for ResNet architectures (requires PyTorch).

In [3]:
model, input_node, output_node = d5nt.create_model('resnet', depth=20, classes=shape[0], batch_size=batch_size)

Removed 21 superfluous nodes in graph


Next, we will add the loss to the end of the model, in order to be able to train the classification problem (the names 'label' and 'loss' are arbitrary):

In [4]:
model.add_operation(loss([output_node, 'label'], 'loss'))

81

## Dataset

Constructing the dataset requires knowing the input and target nodes, so we construct it after the model. We construct both training and validation set at the same time. For training, we also create a sampler that randomizes access by shuffling the data every epoch.

In [5]:
train_set, validation_set = d5ds.load_dataset('cifar10', input_node, 'label')
train_sampler = d5.ShuffleSampler(train_set, batch_size)

Download complete.

unzipping in path: C:\Users\XL\AppData\Local\Temp\cifar10
done!


## Executor

We use PyTorch as our graph executor, constructing it from the model, and set the GPU to be the main device:

In [6]:
from deep500.frameworks import pytorch as d5fw
executor = d5fw.from_model(model, device=d5.GPUDevice())

## Optimizer

We choose the framework-native momentum optimizer, and construct it with our executor and loss node name:

In [7]:
optimizer = d5fw.MomentumOptimizer(executor, learning_rate=0.1, momentum=0.9, loss='loss')

## Training

Everything is ready, now we can train. We add some global metrics to show how to customize them:

In [8]:
d5.test_training(executor, train_sampler, validation_set,
                 optimizer, epochs, batch_size, output_node,
                 metrics=[d5.WallclockTime(reruns=0, avg_over=1)])

Testing: 100%|████████████████████████████████████████| 156/156 [00:09<00:00, 16.85it/s, accuracy=73.2, test_loss=0.81]


WallclockTime: 395.95 seconds


[[395.94624042510986]]

## Adding Data Augmentations into the Mix

Instead of observing the same images shuffled, we can create a dataset sampler that augments the input samples by randomly cropping the images, flipping them, and cutting out parts of them. To do so, we reconstruct a new training Sampler, and load reference implementations of data augmentation:

In [8]:
from deep500.frameworks import reference as d5ref
train_sampler = d5.ShuffleSampler(train_set, batch_size, transformations=[
    d5ref.Crop((32, 32), random_crop=True, padding=(4, 4)),
    d5ref.RandomFlip(),
    d5ref.Cutout(1, 16),
])

In [9]:
executor = d5fw.from_model(model, device=d5.GPUDevice())
optimizer = d5fw.MomentumOptimizer(executor, learning_rate=0.1, momentum=0.9, loss='loss')
d5.test_training(executor, train_sampler, validation_set,
                 optimizer, epochs, batch_size, output_node,
                 metrics=[d5.WallclockTime(reruns=0, avg_over=1)])

Testing: 100%|█████████████████████████████████████████| 156/156 [00:10<00:00, 14.84it/s, accuracy=70, test_loss=0.951]


WallclockTime: 815.92 seconds


[[815.9192576408386]]

Seemingly, the results of this run are worse than without data augmentations. However, running the full training procedure for 90 epochs should yield higher overall generalization.